In [272]:
# copied from

In [2]:
from metrics import get_baseline_data
import numpy as np
import math
import tensorflow
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras import optimizers, metrics
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import RobustScaler

2022-03-04 23:04:11.772227: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-04 23:04:11.774078: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [15]:
data = get_baseline_data("raw_data/preproc_data_rate.csv")
data_wo_date = data.drop(columns="Date")

In [16]:
## Notes for group members:
# Attention: This model is fitted with a regular validation set. We should actually adjust it in a way that the validation
# set consists of a number of randomly selected sequences.
# On March 4, this model was also uploaded to Google Drive.
# To get the model from Google drive, see the corresponding link in the slack group.

def subsample_sequence(data, length): # Return a shorter dataframe with specified length
    last_possible = data.shape[0] - length
    random_start = np.random.randint(0, last_possible)
    data_sample = data[random_start: random_start+length]
    return data_sample

def split_subsample_sequence(data, length): # Return a random sequence of specified length
    data_subsample = subsample_sequence(data, length)
    y_sample = data_subsample.iloc[length-31:]

    X_sample = data_subsample[0:length-31]
    X_sample = X_sample.values
    return np.array(X_sample), np.array(y_sample)

def get_X_y(data, n_sequences, length): # Return a sepcific number of (X,y) samples of specified length for all adm. regions

    X, y = [], []

    for i in range(n_sequences):
        (xi, yi) = split_subsample_sequence(data, length)
        X.append(xi)
        y.append(yi)

    X = np.array(X)
    y = np.array(y)
    return X, y

def get_train_test(data,n_sequences,length): # Return train and test data

    len_ = int(0.8*data.shape[0])
    data_train = data[:len_]
    data_test = data[len_:]

    test_seq = math.floor(n_sequences/4)

    X_train, y_train = get_X_y(data_train, n_sequences, length)
    X_test, y_test = get_X_y(data_test, test_seq, length)

#     X_train = X_train.reshape(X_train.shape[0], X_train.shape[1],1)
#     X_test = X_test.reshape(X_test.shape[0], X_test.shape[1],1)

    return X_train, y_train, X_test, y_test

def model(number_of_sequences, input_sequence_length, number_of_regions, prediction_horizon):
# number_of_sequences = 1000 # number of data sequences for training
# input_sequence_length = 169 # sequence length in training process
# number_of_regions = 30 # number of region sin training process
# prediction_horizon = 31 # number of predicted days by region
#X_train_shape = (number_of_sequences, input_sequence_length, number_of_regions)
#y_train_shape = (number_of_sequences, input_sequence_length, number_of_regions)

    model = models.Sequential()
    model.add(layers.LSTM(40, return_sequences=False, activation="tanh", input_shape = (input_sequence_length, number_of_regions)))
    model.add(layers.RepeatVector(prediction_horizon))
    model.add(layers.LSTM(40, return_sequences=True, activation="tanh"))
    model.add(layers.TimeDistributed(layers.Dense(number_of_regions,"relu")))
    model.compile(loss="mse",
                optimizer="rmsprop")
    return model

def fit_model(X_train, y_train, model):
    es = EarlyStopping(monitor='val_loss', verbose=1, patience=20, restore_best_weights=True)
    hist = model.fit(X_train, y_train, callbacks=[es],epochs = 2000,validation_split =0.3, batch_size=32)
    return hist

In [17]:
_,_, X_test, y_test = get_train_test(data_wo_date,1000,200)

In [19]:
from tensorflow import keras
model_AK = keras.models.load_model('RNN_LSTM')
model_AK.summary()
model_AK.predict(X_test)

Model: "sequential_65"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_110 (LSTM)             (None, 40)                11360     
                                                                 
 repeat_vector_25 (RepeatVec  (None, 31, 40)           0         
 tor)                                                            
                                                                 
 lstm_111 (LSTM)             (None, 31, 40)            12960     
                                                                 
 time_distributed_66 (TimeDi  (None, 31, 30)           1230      
 stributed)                                                      
                                                                 
Total params: 25,550
Trainable params: 25,550
Non-trainable params: 0
_________________________________________________________________


array([[[0.02681525, 0.        , 0.        , ..., 0.05598386,
         0.05453284, 0.02433676],
        [0.04008394, 0.        , 0.        , ..., 0.08577636,
         0.07221656, 0.03931981],
        [0.05084032, 0.        , 0.        , ..., 0.10828082,
         0.085325  , 0.04941237],
        ...,
        [0.08007121, 0.        , 0.        , ..., 0.17520091,
         0.1250845 , 0.0934656 ],
        [0.08008624, 0.        , 0.        , ..., 0.17522833,
         0.1251506 , 0.09353571],
        [0.08010253, 0.        , 0.        , ..., 0.17525017,
         0.12520453, 0.0935932 ]],

       [[0.02441004, 0.        , 0.        , ..., 0.06009176,
         0.04844568, 0.02230829],
        [0.03610854, 0.        , 0.        , ..., 0.09281369,
         0.06158608, 0.03629595],
        [0.04580789, 0.        , 0.        , ..., 0.11744565,
         0.0714482 , 0.04600593],
        ...,
        [0.07088912, 0.        , 0.        , ..., 0.18946515,
         0.1056214 , 0.08943886],
        [0.0